## Web Search Test
The **Web Search Test** ant demonstrates the use of a Web API secured using an API key with an auth header. This sample requires you to create a Web Search (bing) resource in Azure and enter its corresponding API key in [0-AI-settings](0-AI-settings.ipynb)

## First steps
- First, download AntRunLib from Nuget
- At least once, set up the environment using **[0-AI-settings](0-AI-settings.ipynb)**

In [1]:
#r "nuget: AntRunnerLib, 0.8.6"

using AntRunnerLib;
using AntRunnerLib.Identity;
using static AntRunnerLib.ClientUtility;
using System.IO;

#!import config/Settings.cs 

var envVariables = Settings.GetEnvironmentVariables();
foreach (var kvp in envVariables)
{
    Environment.SetEnvironmentVariable(kvp.Key, kvp.Value);
}

var config = AzureOpenAiConfigFactory.Get();
var client = GetOpenAiClient(config);


Installed Packages AntRunnerLib, 0.8.6

## Ensure the Web Search Test assistant exists
The definition of this assistant is located in the ".\AssistantDefinitions\Web Search Test" folder

".\AssistantDefinitions\" is a default path. You can override it by setting the **ASSISTANTS_BASE_FOLDER_PATH** environment variable.

"Web Search Test" contains the following files:
```
│   instructions.md - The instructions
│   manifest.json - The base assistant definition
│
└───OpenAPI
        api.bing.microsoft.com.json - A swagger file defining a tool named 'crawl' that uses web search
        auth.json - See below... 
```        

## auth.json
AntRunnerLib uses the instructions in this file to create the correct auth header(s) for the API. In this case, a header is added to the web request, **Ocp-Apim-Subscription-Key**, the header's value is supplied by an environment variable named, SEARCH_API_KEY. The environment variable name is user defined and ensuring it has a value is the responsibility of the application. 

```json
{
  "hosts": {
    "api.bing.microsoft.com": {
      "auth_type": "service_http",
      "header_name": "Ocp-Apim-Subscription-Key",
      "header_value_env_var": "SEARCH_API_KEY"
    }
  }
}
```

### Explanation
`AssistantUtility.Create` acts based on the contents of a folder structure following a convention laid out in these notebooks. In this case, it copied the contents of instructions.md into the assistant manifest.json. It also parsed the api.bing.microsoft.com swagger which follows an Open API schema into tool defintions for the assitant.

In [2]:
var assistantId = await AssistantUtility.Create("Web Search Test", config);
Console.WriteLine(assistantId)

asst_5y0Ih36HavI1Dw3YjQNvzItY


## Run the Assistant

`output.Dialog` shows the conversation.

In [3]:
var assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "Web Search Test",
    Instructions = "Use python to get todays date and then use it to find things to do in Atlanta today. Be careful to not include events in the past in your final answer.",
};
var output = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
output.Dialog.DisplayAs("text/markdown")


User: Use python to get todays date and then use it to find things to do in Atlanta today. Be careful to not include events in the past in your final answer.

Assistant: I ran this code:
```
from datetime import datetime

# Get today's date
today_date = datetime.now().date()
today_date
```

I called the tool named crawl with these arguments:
```
{
  "q": "things to do in Atlanta on September 24, 2024",
  "count": 5,
  "textFormat": "HTML"
}
```
and got this result:
```
{"queryContext":{"originalQuery":"things to do in Atlanta on September 24, 2024"},"webPages":{"value":[{"name":"Cool Things To Do in September in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta","snippet":"The Sweet Auburn Music Fest, which takes place Sept. 28-29. The festival, located on Auburn Avenue, one of Atlanta\u2019s most iconic streets, offers fun-filled entertainment, live music \u2014 everything from R\u0026amp;B to gospel to alternative favorites \u2014 international food court, national artists, celebrities and vendors of all types.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BSeptember\u002B24%2c\u002B2024\u0026d=4692574351720481\u0026mkt=en-US\u0026setlang=en-US\u0026w=IyKVbCksY9VV74GkIkdoRE4k4vk9MNbJ","thumbnailUrl":"https://www.bing.com/th?id=OIP.m0ENCio7ytS2o9PQRWDCRgHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Events September 2024: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/september/","displayUrl":"https://atlanta-ga.events/september","snippet":"Check Out The Atlanta Events Calendar for September 2024. See the List Of All Current \u0026amp; Upcoming Events at the Lowest Possible Price. ... Sep 24, 2024 6:00 PM . Edwin Raphael . Upcoming Events: 2. Altar at Masquerade - GA . Sep 24, 2024 7:00 PM ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BSeptember\u002B24%2c\u002B2024\u0026d=4854189675467386\u0026mkt=en-US\u0026setlang=en-US\u0026w=ShlkdiIwTMefbkmdL1lCvH4mS1bjztOY"},{"name":"Atlanta Events - September 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html","snippet":"Things to do in Atlanta, GA in September, including upcoming festivals, Labor Day Weekend events, Pride events, neighborhood events, comedy, musicals, races, conventions, shows and more. ... \u2013 Geeked Week 24, September 19 2024 Netflix preview event at The Eastern. More details \u2013 Georgia Latino International Film Festival, September 19-22 ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BSeptember\u002B24%2c\u002B2024\u0026d=4712253893443652\u0026mkt=en-US\u0026setlang=en-US\u0026w=-nsAOlbynJFe4oOvzq1dGpeg_GE4sHjt","thumbnailUrl":"https://www.bing.com/th?id=OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"The Best Things to Do in Atlanta in September","url":"https://unexpectedatlanta.com/the-best-things-to-do-in-atlanta-in-september/","displayUrl":"https://unexpectedatlanta.com/the-best-things-to-do-in-atlanta-in-september","snippet":"Atlanta Events: September 2024 Experience the Best of Atlanta\u2019s Food \u0026amp; History. Try some of the best eats in Atlanta! The #1-rated Grant Park Past \u0026amp; Future Food Walk is a fun walking tour showcasing the best of Atlanta\u2019s food and history. On this tour, you\u2019ll taste eight modern and Southern foods from three outstanding locally owned food stops.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BSeptember\u002B24%2c\u002B2024\u0026d=4781604725337408\u0026mkt=en-US\u0026setlang=en-US\u0026w=L1hd-5qzDrt-bHwGvtgeUoQiW0MCD0-O","thumbnailUrl":"https://www.bing.com/th?id=OIP.BV6NvB-FhOL3LSCX1dcUJwHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"20 Spectacular Things To Do This September In Atlanta - Secret Atlanta","url":"https://secretatlanta.co/things-to-do-september/","displayUrl":"https://secretatlanta.co/things-to-do-september","snippet":"When: Saturday, September 7th from 12pm \u2013 10pm. Thousands of people are expected to head to Morris Brown College in Atlanta this September for the legendary Seafood \u0026amp; Caribbean Music festival! Enjoy tons of live music, DJs, and of course, as much mouthwatering seafood and Caribbean dishes as your heart desires! Don\u2019t miss out on all the fun.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002Bon\u002BSeptember\u002B24%2c\u002B2024\u0026d=4724958405090999\u0026mkt=en-US\u0026setlang=en-US\u0026w=HkmWmkX0emVkqeBkOazQ7cCeg5xgwMQ3","thumbnailUrl":"https://www.bing.com/th?id=OIP.mwxBnPRtbggmsDGaNhGACQHaE8\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"}]}}
```

Here are some events and activities you can enjoy in Atlanta today, September 24, 2024:

1. [Edwin Raphael Concert](https://atlanta-ga.events/september/) - At 6:00 PM at Altar at Masquerade - GA.
2. [Grant Park Past & Future Food Walk](https://unexpectedatlanta.com/the-best-things-to-do-in-atlanta-in-september/) - A fun walking tour showcasing the best of Atlanta’s food and history, featuring eight modern and Southern foods from three locally owned food stops.
3. [20 Spectacular Things To Do This September In Atlanta](https://secretatlanta.co/things-to-do-september/) - Various events including live music, DJs, and mouthwatering seafood and Caribbean dishes.

For more details, you can visit the links provided. Enjoy your time in Atlanta!


In [8]:
output.LastMessage.DisplayAs("text/markdown");

Here are some exciting events and things to do in Atlanta for each of the next four weekends:

**September 28-29, 2024:**
1. **Sweet Auburn Music Fest**: Located on Auburn Avenue, offering live music, international food court, national artists, celebrities, and vendors of all types. [Learn more](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/)
2. **Riverfest**: An annual juried arts and crafts festival in Etowah River Park, Canton, with 150+ artists and artisans. [Learn more](https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html)
3. **East Atlanta Strut**: This iconic street festival celebrates East Atlanta’s culture, art, and quirkiness. [Learn more](https://www.atlantaonthecheap.com/fall-festivals-atlanta/)
4. **Concerts and Shows**: Various concerts and shows happening around Atlanta. [Learn more](https://atlanta-ga.events/september/)

**October 5-6, 2024:**
1. **Festival on Ponce**: Arts festival in Olmsted Linear Park with 125+ vendors, food, drink, and children’s activities. [Learn more](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html)
2. **Candler Park Fall Fest**: Includes an artist market, live acoustic music, rides, a tour of homes, food, and kids activities. [Learn more](https://www.atlantaonthecheap.com/fall-festivals-atlanta/)
3. **Atlanta Fair**: A traditional fair with cotton candy, a Ferris wheel, fishing for goldfish, and more. Located across from the Georgia State University Stadium. [Learn more](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/)
4. **Atlanta Horror Film Festival**: Held at Limelight Theater featuring the best indie horror films. [Learn more](https://www.experism.com/atlanta/october)

**October 12-13, 2024:**
1. **Ellijay Apple Festival**: Spend a day on the farm, pick your own apples, and visit over 100 vendor booths for arts, crafts, and apple-based food items. [Learn more](https://www.atlantaonthecheap.com/fall-festivals-atlanta/)
2. **Atlanta Pride Festival and Parade**: A vibrant event celebrating and supporting the LGBTQ+ community. [Learn more](https://unexpectedatlanta.com/atlanta-festival-calendar-2024/)
3. **Concerts and Shows**: Various concerts and shows happening around Atlanta. [Learn more](https://atlanta-ga.events/october/)
4. **Atlanta Fair**: Continues through October. [Learn more](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/)

**October 19-20, 2024:**
1. **PNC Atlanta 10 Miler & 5K**: Annual runs through Buckhead from Atlantic Station. [Learn more](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html)
2. **Little 5 Points Halloween Festival & Parade**: Features live music, an art and vintage market, ghost tours, a skate park, and food and drink vendors. [Learn more](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/)
3. **The Upside Down**: Event featuring celebrity panels, exclusive photo ops, and autograph sessions from celebrities from Stranger Things. [Learn more](https://www.epiccons.com/events/the-upside-down)
4. **Ellijay Apple Festival**: Continues through the weekend. [Learn more](https://www.atlantaonthecheap.com/fall-festivals-atlanta/)

Feel free to explore these events and make the most of your weekends in Atlanta!

In [5]:
var assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "Web Search Test",
    Instructions = "Use python to get todays date and then use it to find things to do in Atlanta for each of the next four weekends. Be careful to not include events in the past in your final answer. You should perform four searches at once.",
    UseConversationEvaluator = false
};
var output = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
output.Dialog.DisplayAs("text/markdown");


User: Use python to get todays date and then use it to find things to do in Atlanta for each of the next four weekends. Be careful to not include events in the past in your final answer. You should perform four searches at once.

Assistant: I ran this code:
```
from datetime import datetime, timedelta

# Get today's date
today = datetime.today()

# Calculate the dates for the next four weekends (Saturday and Sunday)
weekends = []
for i in range(4):
    saturday = today + timedelta(days=(5 - today.weekday()) + i * 7)
    sunday = saturday + timedelta(days=1)
    weekends.append((saturday.date(), sunday.date()))

weekends
```

I called the tool named crawl with these arguments:
```
{"q": "things to do in Atlanta September 28-29 2024", "answerCount": 5, "textFormat": "HTML"}
```
and got this result:
```
{"queryContext":{"originalQuery":"things to do in Atlanta September 28-29 2024"},"webPages":{"value":[{"name":"Events and Things to Do in Atlanta GA this from 2024-09-28 to 2024-09-29","url":"https://eventcartel.com/events/all-categories/atlanta/from_2024-09-28_to_2024-09-29","displayUrl":"https://eventcartel.com/events/all-categories/atlanta/from_2024-09-28_to_2024-09-29","snippet":"Explore a wide range of events and things to do from 2024-09-28 to 2024-09-29 tailored to offer you the best in entertainment and fun things to do in Atlanta. Whether you\u2019re interested in the latest concerts, theater shows, or other local events happening from 2024-09-28 to 2024-09-29, everything you need is just a few clicks away.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BSeptember\u002B28-29\u002B2024\u0026d=593829212581\u0026mkt=en-US\u0026setlang=en-US\u0026w=ArLA-810ZZkaWFgm_VL8k_3YAMhRPChX"},{"name":"Cool Things To Do in September in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta","snippet":"The Sweet Auburn Music Fest, which takes place Sept. 28-29. The festival, located on Auburn Avenue, one of Atlanta\u2019s most iconic streets, offers fun-filled entertainment, live music \u2014 everything from R\u0026amp;B to gospel to alternative favorites \u2014 international food court, national artists, celebrities and vendors of all types.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BSeptember\u002B28-29\u002B2024\u0026d=4692574351720481\u0026mkt=en-US\u0026setlang=en-US\u0026w=IyKVbCksY9VV74GkIkdoRE4k4vk9MNbJ","thumbnailUrl":"https://www.bing.com/th?id=OIP.m0ENCio7ytS2o9PQRWDCRgHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Events - September 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html","snippet":"Things to do in Atlanta, GA in September, including upcoming festivals, Labor Day Weekend events, Pride events, neighborhood events, comedy, musicals, races, conventions, shows and more. ... \u2013 Riverfest, September 28-29 2024 Annual juried arts and crafts festival in Etowah River Park, Canton, with 150\u002B artists and artisans from across the ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BSeptember\u002B28-29\u002B2024\u0026d=4712253893443652\u0026mkt=en-US\u0026setlang=en-US\u0026w=-nsAOlbynJFe4oOvzq1dGpeg_GE4sHjt","thumbnailUrl":"https://www.bing.com/th?id=OIP.xm-3dn3hYJZHsVVU9WO5qwHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Events September 2024: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/september/","displayUrl":"https://atlanta-ga.events/september","snippet":"Check Out The Atlanta Events Calendar for September 2024. See the List Of All Current \u0026amp; Upcoming Events at the Lowest Possible Price. ... This guide is your gateway to discovering the best things to do in Atlanta, ... Prices from $28 Avg. price ~ $33. 20 tickets remaining! Tickets; Sep. 24. 2024. 7:00 PM Tue. Kehlani. Lakewood Amphitheatre ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BSeptember\u002B28-29\u002B2024\u0026d=4854189675467386\u0026mkt=en-US\u0026setlang=en-US\u0026w=ShlkdiIwTMefbkmdL1lCvH4mS1bjztOY"},{"name":"80\u002B fall festivals \u0026amp; fairs around metro Atlanta: Fun things to do in ...","url":"https://www.atlantaonthecheap.com/fall-festivals-atlanta/","displayUrl":"https://www.atlantaonthecheap.com/fall-festivals-atlanta","snippet":"\u2666 East Atlanta Strut. September 28, 2024 East Atlanta Village FREE. This iconic street festival celebrates East Atlanta\u2019s culture, art, and quirkiness! ... September 29, 2024 Lake Sheryl 4540 Lee Rd SW Snellville, GA 30039. Come for soul food, music genres from jazz to reggae, local vendors and more. Save room for the collard green ice cream!","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BSeptember\u002B28-29\u002B2024\u0026d=5012794228740299\u0026mkt=en-US\u0026setlang=en-US\u0026w=xfpymjtTccXnjjGt9vluoyq1DPm6ofe-","thumbnailUrl":"https://www.bing.com/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"20 Spectacular Things To Do This September In Atlanta - Secret Atlanta","url":"https://secretatlanta.co/things-to-do-september/","displayUrl":"https://secretatlanta.co/things-to-do-september","snippet":"When: Saturday, September 7th from 12pm \u2013 10pm. Thousands of people are expected to head to Morris Brown College in Atlanta this September for the legendary Seafood \u0026amp; Caribbean Music festival! Enjoy tons of live music, DJs, and of course, as much mouthwatering seafood and Caribbean dishes as your heart desires! Don\u2019t miss out on all the fun.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BSeptember\u002B28-29\u002B2024\u0026d=4724958405090999\u0026mkt=en-US\u0026setlang=en-US\u0026w=HkmWmkX0emVkqeBkOazQ7cCeg5xgwMQ3","thumbnailUrl":"https://www.bing.com/th?id=OIP.mwxBnPRtbggmsDGaNhGACQHaE8\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta September 2024: Events, Concerts, Clubs \u0026amp; Things to Do - Experism","url":"https://www.experism.com/atlanta/september","displayUrl":"https://www.experism.com/atlanta/september","snippet":"Sep 12-15, 2024 Historic Fourth Ward Park View on Map. Play Video. The Atlanta Food \u0026amp; Wine Festival highlights the best food, wine, beer \u0026amp; cocktails Atlanta has to offer. This unique event brings in 150\u002B chefs, sommeliers, mixologists and industry insiders for 5 days of mouthwatering and tempting cuisines \u0026amp; special Tasting Tents.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BSeptember\u002B28-29\u002B2024\u0026d=4642636760352967\u0026mkt=en-US\u0026setlang=en-US\u0026w=CrMoojBamU4_Ok9mubm7deAiSBD33s7x","thumbnailUrl":"https://www.bing.com/th?id=OIP.opEpXe5xeSIGAQVvWLFBbAHaEK\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"The Best Things to Do in Atlanta in September","url":"https://unexpectedatlanta.com/the-best-things-to-do-in-atlanta-in-september/","displayUrl":"https://unexpectedatlanta.com/the-best-things-to-do-in-atlanta-in-september","snippet":"Atlanta Events: September 2024 Experience the Best of Atlanta\u2019s Food \u0026amp; History. Try some of the best eats in Atlanta! The #1-rated Grant Park Past \u0026amp; Future Food Walk is a fun walking tour showcasing the best of Atlanta\u2019s food and history. On this tour, you\u2019ll taste eight modern and Southern foods from three outstanding locally owned food stops.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BSeptember\u002B28-29\u002B2024\u0026d=4781604725337408\u0026mkt=en-US\u0026setlang=en-US\u0026w=L1hd-5qzDrt-bHwGvtgeUoQiW0MCD0-O","thumbnailUrl":"https://www.bing.com/th?id=OIP.BV6NvB-FhOL3LSCX1dcUJwHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Things to Do in Atlanta in 2024: Events, festivals, and exhibits ...","url":"https://www.thrillist.com/events/atlanta/events-calendar-in-atlanta-this-year","displayUrl":"https://www.thrillist.com/events/atlanta/events-calendar-in-atlanta-this-year","snippet":"23 Things You Have to Do in Atlanta This Year, From One Musicfest to Homecoming ... September 28\u201329, 2024 Old Fourth Ward One of the city\u2019s most anticipated events is back in September. The ...","thumbnailUrl":"https://www.bing.com/th?id=OIP.WUHCPl7_RpepE3pSEdAEMgHaE8\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"35\u002B Best Things to Do in Atlanta in 2024 - Unexpected Atlanta","url":"https://unexpectedatlanta.com/best-things-to-do-in-atlanta/","displayUrl":"https://unexpectedatlanta.com/best-things-to-do-in-atlanta","snippet":"1. Visit the National Parks on a Fee-Free Day. First on our list of the best things to do in Atlanta in 2024, check out America\u2019s National Park System\u2019s fee-free days this year. On these special days, you can visit any of the United States\u2019 400\u002B National Parks for free, including places in and around Georgia.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BSeptember\u002B28-29\u002B2024\u0026d=4885792044096348\u0026mkt=en-US\u0026setlang=en-US\u0026w=SRiegEmhRGKtzRRUG028yHS95G2gOIKx","thumbnailUrl":"https://www.bing.com/th?id=OIP.HwzfdFE6-tSHfhoFlTxXKQHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"}]}}
```

I called the tool named crawl with these arguments:
```
{"q": "things to do in Atlanta October 5-6 2024", "answerCount": 5, "textFormat": "HTML"}
```
and got this result:
```
{"queryContext":{"originalQuery":"things to do in Atlanta October 5-6 2024"},"webPages":{"value":[{"name":"Atlanta Events - October 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html","snippet":"Wrestling event at State Farm Arena. More details. \u2013 Festival on Ponce, October 5-6 2024. Twice-annual arts festival in Olmsted Linear Park, with 125\u002B vendors of fine art, folk art and outsider art, plus food and drink, and children\u2019s activities. More details. \u2013 Candler Park Fall Fest, October 5-6 2024.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B5-6\u002B2024\u0026d=4907344187564116\u0026mkt=en-US\u0026setlang=en-US\u0026w=DKKAeoP_3oe3mkDLUvcm_m1afK8iBTpt"},{"name":"Atlanta Events October 2024: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/october/","displayUrl":"https://atlanta-ga.events/october","snippet":"Explore our comprehensive calendar for all the current and upcoming events in Atlanta, GA, for October 2024. This guide is your gateway to discovering the best things to do in Atlanta, from Concerts and Theater Shows to Sports events and music festivals. You\u2019ll find a detailed list of all the most engaging and exciting activities happening throughout the month.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B5-6\u002B2024\u0026d=4898908879865838\u0026mkt=en-US\u0026setlang=en-US\u0026w=vC_zzFLZ91uzxHogtJy0oBuKycEPAm_-"},{"name":"80\u002B fall festivals \u0026amp; fairs around metro Atlanta: Fun things to do in ...","url":"https://www.atlantaonthecheap.com/fall-festivals-atlanta/","displayUrl":"https://www.atlantaonthecheap.com/fall-festivals-atlanta","snippet":"October 5 and 6, 2024 Candler Park Atlanta FREE. This popular street festival includes an artist market, live acoustic music on two stages, rides and special attractions, a tour of homes, food, kids activities, and more. ... October 5 and 6, 2024 Olmsted Linear Parks Atlanta FREE.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B5-6\u002B2024\u0026d=5012794228740299\u0026mkt=en-US\u0026setlang=en-US\u0026w=xfpymjtTccXnjjGt9vluoyq1DPm6ofe-","thumbnailUrl":"https://www.bing.com/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Cool Things To Do in October in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta","snippet":"Corn Dogs, Rides and Whack a Mole. The Atlanta Fair brings the best of a traditional fair to Atlanta with everything from cotton candy, a Ferris wheel, fishing for goldfish, a tilt-a-whirl and a carousel. The fair, located across from the Georgia State University Stadium, will be in town Oct. 4-Nov. 3. Read More.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B5-6\u002B2024\u0026d=4714289705734711\u0026mkt=en-US\u0026setlang=en-US\u0026w=6a-yO_DfCUxfz-dx9C91WDDECk49pD7V","thumbnailUrl":"https://www.bing.com/th?id=OIP.4jf9xqKu3QRk_XhZ3J7DYQHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Fall festivals in metro Atlanta and North Georgia | 2024","url":"https://www.fox5atlanta.com/news/fall-festivals-metro-atlanta-north-georgia-2024","displayUrl":"https://www.fox5atlanta.com/news/fall-festivals-metro-atlanta-north-georgia-2024","snippet":"When: 10 a.m. to 5 p.m. Oct. 5 and 11 a.m. to 5 p.m. Oct. 6 Where : Olmstead Linear Park, Atlanta What : Arts and crafts festival set in the historic Olmsted Linear Park with over 125 displays ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B5-6\u002B2024\u0026d=2173865470069\u0026mkt=en-US\u0026setlang=en-US\u0026w=jxnOnRj04ciRcKNV8UVNHXKEd0PX1nap","thumbnailUrl":"https://www.bing.com/th?id=OIF.XT1OwgQKRVd2itXsxBJl%2bA\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"26 Thrilling Things To Do In Atlanta This October - Secret Atlanta","url":"https://secretatlanta.co/things-to-do-october/","displayUrl":"https://secretatlanta.co/things-to-do-october","snippet":"Without further ado, here are 26 incredible things to do in Atlanta this October! 1. ... September 20, 2024. 18 Frightfully Fun Halloween Events Happening In Atlanta. September 20, 2024. This Creepy Corn Maze In Buford Comes Alive With Spooks \u0026amp; Haunts At Sunset.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B5-6\u002B2024\u0026d=4673096676868625\u0026mkt=en-US\u0026setlang=en-US\u0026w=BDZ6Krx8ATVNFCQACSuK0PesDCkO7vK5","thumbnailUrl":"https://www.bing.com/th?id=OIP.oVWYpGkEV4zoUw2uzcrcYgHaD6\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta October 2024: Events, Concerts, Clubs \u0026amp; Things to Do - Experism","url":"https://www.experism.com/atlanta/october","displayUrl":"https://www.experism.com/atlanta/october","snippet":"Atlanta Horror Film Festival. Oct 4-6, 2024 Limelight Theater View on Map. You are cordially invited to the 18th Annual Atlanta Horror Film Festival held at Limelight Theater from October 4 - 6, 2024. Don\u0026#39;t miss this spooktacular collection of the best indie horror films from across the globe, including world premieres, short films, animation ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B5-6\u002B2024\u0026d=5050860516094956\u0026mkt=en-US\u0026setlang=en-US\u0026w=OllQT5kZyN343ZtIpGRATFj2b4uP5Oth","thumbnailUrl":"https://www.bing.com/th?id=OIP.8x_jPSRdmgimGRTNJq3pngAAAA\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Festival Calendar: 2024","url":"https://unexpectedatlanta.com/atlanta-festival-calendar-2024/","displayUrl":"https://unexpectedatlanta.com/atlanta-festival-calendar-2024","snippet":"Atlanta Jazz Festival \u2013 May 25-27, 2024. Held in Piedmont Park, the Atlanta Jazz Festival is an annual celebration of music, culture, and the art of Jazz. Free to all who join, this Memorial Day Weekend event draws fans, professional musicians, and everyone in between, for a 3-day-long festival showcasing 15\u002B artists.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B5-6\u002B2024\u0026d=4520195830737555\u0026mkt=en-US\u0026setlang=en-US\u0026w=Y_UYHSuQi3QHjG8exIRi2zfYmaaF9rTA","thumbnailUrl":"https://www.bing.com/th?id=OIP.8LvXI6tOuWvo9KZdL-C3MwHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"35\u002B Best Things to Do in Atlanta in 2024 - Unexpected Atlanta","url":"https://unexpectedatlanta.com/best-things-to-do-in-atlanta/","displayUrl":"https://unexpectedatlanta.com/best-things-to-do-in-atlanta","snippet":"1. Visit the National Parks on a Fee-Free Day. First on our list of the best things to do in Atlanta in 2024, check out America\u2019s National Park System\u2019s fee-free days this year. On these special days, you can visit any of the United States\u2019 400\u002B National Parks for free, including places in and around Georgia.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B5-6\u002B2024\u0026d=4885792044096348\u0026mkt=en-US\u0026setlang=en-US\u0026w=SRiegEmhRGKtzRRUG028yHS95G2gOIKx","thumbnailUrl":"https://www.bing.com/th?id=OIP.HwzfdFE6-tSHfhoFlTxXKQHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"The Best Things to Do in Atlanta in October","url":"https://unexpectedatlanta.com/the-best-things-to-do-in-atlanta-in-october/","displayUrl":"https://unexpectedatlanta.com/the-best-things-to-do-in-atlanta-in-october","snippet":"Atlanta Events in October 2023 Cozy Up with a Biscuit-Making \u0026amp; Food Tour Experience \u2013 One of our Favorite Atlanta October Events! This Historic Market Food Tour \u002B Biscuit Class is the perfect way to cozy up for fall! During your guided experience, you\u2019ll try 11 delicious tastings from 5 local food vendors in Atlanta\u2019s oldest market, the Sweet Auburn Curb Market.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B5-6\u002B2024\u0026d=4733754493195045\u0026mkt=en-US\u0026setlang=en-US\u0026w=wJ10aadQT5xoqbb3_3asNbrAJ_n6Hq4i","thumbnailUrl":"https://www.bing.com/th?id=OIP.lEMypzNNWpS-6tdPg0Hm1gHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"}]}}
```

I called the tool named crawl with these arguments:
```
{"q": "things to do in Atlanta October 12-13 2024", "answerCount": 5, "textFormat": "HTML"}
```
and got this result:
```
{"queryContext":{"originalQuery":"things to do in Atlanta October 12-13 2024"},"webPages":{"value":[{"name":"Atlanta Events - October 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html","snippet":"Events From October 1 To The Weekend Of October 5-6 2024. \u2192 See September events. \u2192 For concerts this week, see Atlanta concerts in October 2024. \u2013 Daddy Long Legs, September 13 - October 6 2024. Musical at The Legacy Theatre, Tyrone. More details.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B12-13\u002B2024\u0026d=4907344187564116\u0026mkt=en-US\u0026setlang=en-US\u0026w=DKKAeoP_3oe3mkDLUvcm_m1afK8iBTpt"},{"name":"Atlanta Events October 2024: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/october/","displayUrl":"https://atlanta-ga.events/october","snippet":"Explore our comprehensive calendar for all the current and upcoming events in Atlanta, GA, for October 2024. This guide is your gateway to discovering the best things to do in Atlanta, ... Prices from $13 Avg. price ~ $66. Available Tickets: 1417. Tickets; Oct. 02. 2024. 7:30 PM Wed.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B12-13\u002B2024\u0026d=4898908879865838\u0026mkt=en-US\u0026setlang=en-US\u0026w=vC_zzFLZ91uzxHogtJy0oBuKycEPAm_-"},{"name":"80\u002B fall festivals \u0026amp; fairs around metro Atlanta: Fun things to do in ...","url":"https://www.atlantaonthecheap.com/fall-festivals-atlanta/","displayUrl":"https://www.atlantaonthecheap.com/fall-festivals-atlanta","snippet":"October 12 and 13, 2024 October 19 and 20, 2024 Ellijay, GA . This is one of the few festivals outside metro Atlanta we include. It takes place in North Georgia, over two weekends in October. Spend a day on the farm, pick your own apples, visit over 100 vendor booths for arts, crafts, and apple-based food items. Well worth the drive!","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B12-13\u002B2024\u0026d=5012794228740299\u0026mkt=en-US\u0026setlang=en-US\u0026w=xfpymjtTccXnjjGt9vluoyq1DPm6ofe-","thumbnailUrl":"https://www.bing.com/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Cool Things To Do in October in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta","snippet":"Corn Dogs, Rides and Whack a Mole. The Atlanta Fair brings the best of a traditional fair to Atlanta with everything from cotton candy, a Ferris wheel, fishing for goldfish, a tilt-a-whirl and a carousel. The fair, located across from the Georgia State University Stadium, will be in town Oct. 4-Nov. 3. Read More.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B12-13\u002B2024\u0026d=4714289705734711\u0026mkt=en-US\u0026setlang=en-US\u0026w=6a-yO_DfCUxfz-dx9C91WDDECk49pD7V","thumbnailUrl":"https://www.bing.com/th?id=OIP.4jf9xqKu3QRk_XhZ3J7DYQHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Festival Calendar: 2024","url":"https://unexpectedatlanta.com/atlanta-festival-calendar-2024/","displayUrl":"https://unexpectedatlanta.com/atlanta-festival-calendar-2024","snippet":"This experience was named by Conde Nast as one of the best things to do in Atlanta! February Festivals in Atlanta Atlanta Winter Beer Festival \u2013 February 10, 2024 ... October 12-13, 2024. The Atlanta Pride Festival and Parade is a vibrant event that celebrates and supports the LGBTQ\u002B community. By attending, be a part of a joyful and ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B12-13\u002B2024\u0026d=4520195830737555\u0026mkt=en-US\u0026setlang=en-US\u0026w=Y_UYHSuQi3QHjG8exIRi2zfYmaaF9rTA","thumbnailUrl":"https://www.bing.com/th?id=OIP.8LvXI6tOuWvo9KZdL-C3MwHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"26 Thrilling Things To Do In Atlanta This October - Secret Atlanta","url":"https://secretatlanta.co/things-to-do-october/","displayUrl":"https://secretatlanta.co/things-to-do-october","snippet":"Without further ado, here are 26 incredible things to do in Atlanta this October! 1. ... Check out our full list of great Atlanta hikes here! 12. ... September 20, 2024. 18 Frightfully Fun Halloween Events Happening In Atlanta. September 20, 2024 ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B12-13\u002B2024\u0026d=4673096676868625\u0026mkt=en-US\u0026setlang=en-US\u0026w=BDZ6Krx8ATVNFCQACSuK0PesDCkO7vK5","thumbnailUrl":"https://www.bing.com/th?id=OIP.oVWYpGkEV4zoUw2uzcrcYgHaD6\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta October 2024: Events, Concerts, Clubs \u0026amp; Things to Do - Experism","url":"https://www.experism.com/atlanta/october","displayUrl":"https://www.experism.com/atlanta/october","snippet":"Things to do in Atlanta October Atlanta Nightclub Events in October Atlanta Sporting Events, Concerts \u0026amp; Show Tickets in October October Event Calendar The Lion King. Oct 2-20, 2024 Fabulous Fox Theatre View on Map. ... Oct 12-13, 2024 Piedmont Park View on Map. Play Video.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B12-13\u002B2024\u0026d=5050860516094956\u0026mkt=en-US\u0026setlang=en-US\u0026w=OllQT5kZyN343ZtIpGRATFj2b4uP5Oth","thumbnailUrl":"https://www.bing.com/th?id=OIP.8x_jPSRdmgimGRTNJq3pngAAAA\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Things to Do in Atlanta in 2024: Events, festivals, and exhibits ...","url":"https://www.thrillist.com/events/atlanta/events-calendar-in-atlanta-this-year","displayUrl":"https://www.thrillist.com/events/atlanta/events-calendar-in-atlanta-this-year","snippet":"23 Things You Have to Do in Atlanta This Year, From One Musicfest to Homecoming ... By Okla Jones. Updated on July 9, 2024 at 12:40 PM. ... October 11\u201313, 2024 Various locations","thumbnailUrl":"https://www.bing.com/th?id=OIP.WUHCPl7_RpepE3pSEdAEMgHaE8\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Explore Atlanta Events for Today, This Week or Weekend","url":"https://discoveratlanta.com/events/main/","displayUrl":"https://discoveratlanta.com/events/m","snippet":"Explore the Atlanta events calendar for things to do today, this week or this weekend. Find upcoming festivals, seasonal events and more! ... Elevate Atlanta Every weekend until 10/13/2024. Little Shop of Horrors 9/23/24 \u2013 9/29/24. Placita Latina Exhibition ... Multiple dates until 10/12/2024. Roberta Lea Live 10/5/24. Strand Ole Opry: Fa-BOO ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B12-13\u002B2024\u0026d=4819881473496114\u0026mkt=en-US\u0026setlang=en-US\u0026w=zvJBiG0Dg4eP1GMx_bJv9FTM8ziERJGY","thumbnailUrl":"https://www.bing.com/th?id=OIP.FkxSV6JbrWatsFMM3_8dIAHaEh\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Concerts In Atlanta, October 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-concerts-october.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-concerts-october.html","snippet":"Concerts On Friday October 11 2024. \u2013 Lake Street Dive at The Eastern. More details. \u2013 Aerosmith: Peace Out The Farewell Tour at State Farm Arena. More details. \u2013 The Black Jacket Symphony: Performing Led Zeppelin\u2019s \u2018IV\u2019. Tribute concert at Atlanta Symphony Hall. More details. \u2013 Loveless at The Masquerade.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B12-13\u002B2024\u0026d=4991061691999650\u0026mkt=en-US\u0026setlang=en-US\u0026w=I17XSEQ6_0vdrEf4HxlkWHNc_Lnct5jK"}]}}
```

I called the tool named crawl with these arguments:
```
{"q": "things to do in Atlanta October 19-20 2024", "answerCount": 5, "textFormat": "HTML"}
```
and got this result:
```
{"queryContext":{"originalQuery":"things to do in Atlanta October 19-20 2024"},"webPages":{"value":[{"name":"Atlanta Events - October 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html","snippet":"Things to do in Atlanta, GA in October, including upcoming festivals, fall and Halloween events, Pride events, comedy, musicals, plays, races, conventions, neighborhood events, cultural events, and more. ... \u2013 PNC Atlanta 10 Miler \u0026amp; 5K, October 19-20 2024 Annual 10-mile, 5K and 1-mile youth runs through Buckhead, from Atlantic Station.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B19-20\u002B2024\u0026d=4907344187564116\u0026mkt=en-US\u0026setlang=en-US\u0026w=DKKAeoP_3oe3mkDLUvcm_m1afK8iBTpt"},{"name":"Atlanta Events October 2024: Concerts, Shows, Sports...","url":"https://atlanta-ga.events/october/","displayUrl":"https://atlanta-ga.events/october","snippet":"Check Out The Atlanta Events Calendar for October 2024: Current \u0026amp; Upcoming Events \u002B Best \u0026amp; Fun Thing To Do in the City. ... 19 tickets remaining! Tickets; Oct. 02. 2024. 8:00 PM Wed. The Hives. The Tabernacle - GA | Capacity: 2600. 30303, 152 Luckie St, Atlanta, GA, US","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B19-20\u002B2024\u0026d=4898908879865838\u0026mkt=en-US\u0026setlang=en-US\u0026w=vC_zzFLZ91uzxHogtJy0oBuKycEPAm_-"},{"name":"Cool Things To Do in October in Atlanta","url":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/","displayUrl":"https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta","snippet":"Cool Things To Do in October in Atlanta. By Mary Welch. Published September 13, 2024 Last Updated September 19, 2024. ... The bohemian neighborhood sponsors a Halloween Festival on Oct. 19 and 20 that features live music from the city\u2019s best underground acts, an art and vintage market, ghost tours, a skate park, and food and drink vendors ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B19-20\u002B2024\u0026d=4714289705734711\u0026mkt=en-US\u0026setlang=en-US\u0026w=6a-yO_DfCUxfz-dx9C91WDDECk49pD7V","thumbnailUrl":"https://www.bing.com/th?id=OIP.4jf9xqKu3QRk_XhZ3J7DYQHaD4\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"80\u002B fall festivals \u0026amp; fairs around metro Atlanta: Fun things to do in ...","url":"https://www.atlantaonthecheap.com/fall-festivals-atlanta/","displayUrl":"https://www.atlantaonthecheap.com/fall-festivals-atlanta","snippet":"October 12 and 13, 2024 October 19 and 20, 2024 Ellijay, GA . This is one of the few festivals outside metro Atlanta we include. It takes place in North Georgia, over two weekends in October. Spend a day on the farm, pick your own apples, visit over 100 vendor booths for arts, crafts, and apple-based food items. Well worth the drive!","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B19-20\u002B2024\u0026d=5012794228740299\u0026mkt=en-US\u0026setlang=en-US\u0026w=xfpymjtTccXnjjGt9vluoyq1DPm6ofe-","thumbnailUrl":"https://www.bing.com/th?id=OIP.w0-v0ll6eXl6tJfBd5-ovwHaEV\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"The Upside Down | October 19-20 | Epic Events and Entertainment - Epic Cons","url":"https://www.epiccons.com/events/the-upside-down","displayUrl":"https://www.epiccons.com/events/the-upside-down","snippet":"Experience The Upside Down event on October 19-20, 2024, featuring celebrity panels, exclusive photo ops, and autograph sessions from celebrities from Stranger Things! Dive into the world of your favorite shows and meet fellow fans! ... The Upside Down (October 19-20) I Was Feeling Festive 2 (December 6-8) 2025 Events. I Was Feeling Epic (April ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B19-20\u002B2024\u0026d=4700464207900499\u0026mkt=en-US\u0026setlang=en-US\u0026w=T_o7uCc0DGNZhmTFjyQoH8RhyWcnYm_y","thumbnailUrl":"https://www.bing.com/th?id=OIP.Wm4-6OSBCztRSKoEZnMp7AHaEL\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Festival Calendar: 2024","url":"https://unexpectedatlanta.com/atlanta-festival-calendar-2024/","displayUrl":"https://unexpectedatlanta.com/atlanta-festival-calendar-2024","snippet":"Atlanta Pride Festival and Parade \u2013 October 12-13, 2024. The Atlanta Pride Festival and Parade is a vibrant event that celebrates and supports the LGBTQ\u002B community. By attending, be a part of a joyful and inclusive atmosphere where diversity, equality, and love are celebrated. Little 5 Points Halloween Festival \u0026amp; Parade \u2013 October 19-20, 2024","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B19-20\u002B2024\u0026d=4520195830737555\u0026mkt=en-US\u0026setlang=en-US\u0026w=Y_UYHSuQi3QHjG8exIRi2zfYmaaF9rTA","thumbnailUrl":"https://www.bing.com/th?id=OIP.8LvXI6tOuWvo9KZdL-C3MwHaHa\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta October 2024: Events, Concerts, Clubs \u0026amp; Things to Do - Experism","url":"https://www.experism.com/atlanta/october","displayUrl":"https://www.experism.com/atlanta/october","snippet":"Atlanta Horror Film Festival. Oct 4-6, 2024 Limelight Theater View on Map. You are cordially invited to the 18th Annual Atlanta Horror Film Festival held at Limelight Theater from October 4 - 6, 2024. Don\u0026#39;t miss this spooktacular collection of the best indie horror films from across the globe, including world premieres, short films, animation ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B19-20\u002B2024\u0026d=5050860516094956\u0026mkt=en-US\u0026setlang=en-US\u0026w=OllQT5kZyN343ZtIpGRATFj2b4uP5Oth","thumbnailUrl":"https://www.bing.com/th?id=OIP.8x_jPSRdmgimGRTNJq3pngAAAA\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta Events \u2013 Upcoming Festivals in Atlanta 2024 - Go South Atlanta","url":"https://gosouthatlanta.com/things-to-do/events/atlanta-festivals.html","displayUrl":"https://gosouthatlanta.com/things-to-do/events/atlanta-festivals.html","snippet":"November 2024. \u2013 Native American Festival and Pow Wow, October 31 - November 3 2024. Big annual festival of Native American heritage at Stone Mountain Park, with live traditional dance, music and storytelling performances, craft demonstrations, wildlife presentations, arts and crafts and more. More details.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B19-20\u002B2024\u0026d=4630752588082363\u0026mkt=en-US\u0026setlang=en-US\u0026w=GvBienXl3Sw50qWfsSic_uix0GcCbDYr","thumbnailUrl":"https://www.bing.com/th?id=OIP.1kWLxDO_JldjYbZyTlfSrQAAAA\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"},{"name":"Atlanta in October | Concerts, Festivals, Tickets \u0026amp; Tour Dates - Songkick","url":"https://www.songkick.com/metro-areas/4120-us-atlanta/october-2024","displayUrl":"https://www.songkick.com/metro-areas/4120-us-atlanta/october-2024","snippet":"Buy tickets for every upcoming concert, festival, gig and tour date taking place in Atlanta in October. Atlanta concerts. Atlanta concerts Atlanta concerts. Blockhead Aisle 5; Boogie T Believe Music Hall; ... Fri 20 Sep 2024 District Atlanta Atlanta, GA, US. Eprom. Sat 02 Nov 2024 Terminal West Atlanta, ... Tue 19 Nov 2024 The Masquerade - Hell ...","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B19-20\u002B2024\u0026d=4726551829817777\u0026mkt=en-US\u0026setlang=en-US\u0026w=GyNsJVWjOONlYyuTqLMEVsVddRXByGfo"},{"name":"The best things to do in October, around metro Atlanta","url":"https://www.atlantaonthecheap.com/october-things-to-do-atlanta/","displayUrl":"https://www.atlantaonthecheap.com/october-things-to-do-atlanta","snippet":"There are no less than FOUR fairs this month \u2014 click thru the links below for dates and details, including free admission days and ride discounts. North Georgia State Fair in Marietta (closes Oct. 1st) Georgia State Fair at Atlanta Motor Speedway. Atlanta Fair at Turner Field. Cumming Country Fair \u0026amp; Festival.","cachedPageUrl":"http://cc.bingj.com/cache.aspx?q=things\u002Bto\u002Bdo\u002Bin\u002BAtlanta\u002BOctober\u002B19-20\u002B2024\u0026d=4836271073722834\u0026mkt=en-US\u0026setlang=en-US\u0026w=tejb6ZkvzOSXSA6GBXhbI5iLI191LzT_","thumbnailUrl":"https://www.bing.com/th?id=OIP.yagw3m-94_qMguEivVZ9hQHaEV\u0026w=80\u0026h=80\u0026c=1\u0026pid=5.1"}]},"images":{"value":[{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.cGQLTADJ7EIEL8n56X81XwHaLH\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/best-things-to-do-in-Atlanta-GA-1.jpg","hostPageUrl":"https://janqkarlotta.pages.dev/rmueufk-things-to-do-in-atlanta-october-2024-uzbmkjn/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.gpxEMgIaDnDSvyMtPJjjdgHaE6\u0026pid=Api","contentUrl":"https://wowtravel.me/wp-content/uploads/2022/06/Top-10-Things-to-Do-in-Atlanta.jpg","hostPageUrl":"https://wowtravel.me/things-to-do-in-atlanta-this-week/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.IqznTH1ocAOkn-XVKO4tyQAAAA\u0026pid=Api","contentUrl":"https://i.pinimg.com/originals/f5/05/8e/f5058e44214459ab6506002aee8aa99f.png","hostPageUrl":"https://www.pinterest.com/pin/101-best-things-to-do-in-atlanta-georgia-the-ultimate-attractions-guide--347621664991790215/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.fLae3sxHYp3BVcZ5UZwd8gHaLM\u0026pid=Api","contentUrl":"https://www.thebrokebackpacker.com/wp-content/uploads/2019/10/things-to-do-atlanta-pin-1.jpg","hostPageUrl":"https://www.thebrokebackpacker.com/things-to-do-in-atlanta-georgia/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.Tg7nOHaEU_re0ukKIjO-QwHaG1\u0026pid=Api","contentUrl":"https://uphomes.com/storage/uploads/aVxdjP6v0zGPok5GHYOee8lHRMVN0GMFiNomqe5s.png","hostPageUrl":"https://uphomes.com/blog/things-to-do-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.lEMypzNNWpS-6tdPg0Hm1gHaHa\u0026pid=Api","contentUrl":"https://unexpectedatlanta.com/wp-content/uploads/2023/09/The-Best-Things-to-Do-in-Atlanta-in-October.png","hostPageUrl":"https://unexpectedatlanta.com/the-best-things-to-do-in-atlanta-in-october/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.M9nQL4kFM4rwNQiRkhAekAHaEo\u0026pid=Api","contentUrl":"https://travellersworldwide.com/wp-content/uploads/2022/05/shutterstock_1039281055.jpg","hostPageUrl":"https://travellersworldwide.com/best-things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.Fe-J-AR_X-7c4ZeUHVaHwQAAAA\u0026pid=Api","contentUrl":"https://www.acaponeconnection.com/wp-content/uploads/2022/05/All-the-Fun-Things-to-Do-in-Atlanta-Georgia-1.jpg","hostPageUrl":"https://www.acaponeconnection.com/all-the-fun-things-to-do-in-atlanta-georgia/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.JIWcXtQtpq7A0DXTjwLQiwHaLG\u0026pid=Api","contentUrl":"https://i.pinimg.com/736x/fc/4a/d2/fc4ad281802906d1cfde53017a3e4dcf.jpg","hostPageUrl":"https://www.pinterest.com/pin/744079169673803460/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.Yac324FtG-dH_KY8XaUTZQHaME\u0026pid=Api","contentUrl":"https://www.frostandsun.com/wp-content/uploads/2017/11/things-to-do-in-atlanta-pin-me-1-628x1024.jpg.webp","hostPageUrl":"https://www.frostandsun.com/things-to-do-in-atlanta/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.Agse3pS4C61ss3RxvVgySwHaEK\u0026pid=Api","contentUrl":"https://wirksmoving.com/wp-content/uploads/2020/04/Things-to-do-in-Atlanta-with-Kids-1536x864.png","hostPageUrl":"https://wirksmoving.com/blog/things-to-do-in-atlanta-ga-with-kids/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.OD8mHt8EuAz_-Rm6pRiOFwHaEK\u0026pid=Api","contentUrl":"https://i.ytimg.com/vi/2ufhW2kEYro/maxresdefault.jpg","hostPageUrl":"https://www.youtube.com/watch?v=2ufhW2kEYro"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.wxOD0OTZ89VvVuQNnfz51wHaL0\u0026pid=Api","contentUrl":"https://i.pinimg.com/originals/7d/b9/cb/7db9cb75f1804a65a7512408c3763aea.jpg","hostPageUrl":"https://www.pinterest.com/pin/42432421477082343/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.zsUlule-ACLeXXDRofPdiAAAAA\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/things-to-do-in-Atlanta-2.jpg","hostPageUrl":"https://www.busytourist.com/things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.Ctt7cutpo0kduVop1QcE-AAAAA\u0026pid=Api","contentUrl":"https://res.cloudinary.com/atlanta/images/f_auto,q_auto/w_2560,h_1565/v1601396905/newAtlanta.com/Atlanta_OneMusicFest_COP-scaled/Atlanta_OneMusicFest_COP-scaled.jpg","hostPageUrl":"https://celestinewtansy.pages.dev/pbtbvb-atlanta-calendar-of-events-2024-ypvllt/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.U9xvQMbjg9T0YiriayftJAHaLH\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/Things-To-Do-In-Atlanta-pin1.jpg","hostPageUrl":"https://www.busytourist.com/things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.eFJevefuy_77CA07mDrb7wHaEK\u0026pid=Api","contentUrl":"https://i.ytimg.com/vi/KfyduILpJO8/maxresdefault.jpg","hostPageUrl":"https://www.youtube.com/watch?v=KfyduILpJO8"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.3DCmfuvb9PKm59kY35gOowHaD5\u0026pid=Api","contentUrl":"https://www.accessatlanta.com/resizer/st3ipUHpjhivT85uXQnam1p9Dk8=/1280x673/smart/cloudfront-us-east-1.images.arcpublishing.com/ajc/5SITMG7WA5BB3FUUWKHED4CF4U.jpg","hostPageUrl":"https://www.accessatlanta.com/atlanta-things-to-do/10-things-to-do-in-atlanta-this-weekend/VNXLGWBIMFBNNJUH5VYLLFZQGA/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.H1JPxx3mkr-XvYjeXre46AHaLH\u0026pid=Api","contentUrl":"https://www.busytourist.com/wp-content/uploads/2020/07/best-things-to-do-in-Atlanta-GA-1-2.jpg","hostPageUrl":"https://www.busytourist.com/things-to-do-in-atlanta-ga/"},{"thumbnailUrl":"https://tse1.mm.bing.net/th?id=OIP.TNv2qzXsSFvw43t4zu0gGgHaE8\u0026pid=Api","contentUrl":"https://www.travelandleisure.com/thmb/3NRm4q9Qg2bdUqXvenPBVeIloV4=/1500x0/filters:no_upscale():max_bytes(150000):strip_icc()/atlanta-skyline-TODOATL0122-195afa04632e43f5be27ef2dcc94ca3b.jpg","hostPageUrl":"https://www.travelandleisure.com/attractions/best-things-to-do-in-atlanta-georgia"}]}}
```

Here are some exciting events and things to do in Atlanta for each of the next four weekends:

**September 28-29, 2024:**
1. **Sweet Auburn Music Fest**: Located on Auburn Avenue, offering live music, international food court, national artists, celebrities, and vendors of all types. [Learn more](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/)
2. **Riverfest**: An annual juried arts and crafts festival in Etowah River Park, Canton, with 150+ artists and artisans. [Learn more](https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html)
3. **East Atlanta Strut**: This iconic street festival celebrates East Atlanta’s culture, art, and quirkiness. [Learn more](https://www.atlantaonthecheap.com/fall-festivals-atlanta/)
4. **Concerts and Shows**: Various concerts and shows happening around Atlanta. [Learn more](https://atlanta-ga.events/september/)

**October 5-6, 2024:**
1. **Festival on Ponce**: Arts festival in Olmsted Linear Park with 125+ vendors, food, drink, and children’s activities. [Learn more](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html)
2. **Candler Park Fall Fest**: Includes an artist market, live acoustic music, rides, a tour of homes, food, and kids activities. [Learn more](https://www.atlantaonthecheap.com/fall-festivals-atlanta/)
3. **Atlanta Fair**: A traditional fair with cotton candy, a Ferris wheel, fishing for goldfish, and more. Located across from the Georgia State University Stadium. [Learn more](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/)
4. **Atlanta Horror Film Festival**: Held at Limelight Theater featuring the best indie horror films. [Learn more](https://www.experism.com/atlanta/october)

**October 12-13, 2024:**
1. **Ellijay Apple Festival**: Spend a day on the farm, pick your own apples, and visit over 100 vendor booths for arts, crafts, and apple-based food items. [Learn more](https://www.atlantaonthecheap.com/fall-festivals-atlanta/)
2. **Atlanta Pride Festival and Parade**: A vibrant event celebrating and supporting the LGBTQ+ community. [Learn more](https://unexpectedatlanta.com/atlanta-festival-calendar-2024/)
3. **Concerts and Shows**: Various concerts and shows happening around Atlanta. [Learn more](https://atlanta-ga.events/october/)
4. **Atlanta Fair**: Continues through October. [Learn more](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/)

**October 19-20, 2024:**
1. **PNC Atlanta 10 Miler & 5K**: Annual runs through Buckhead from Atlantic Station. [Learn more](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html)
2. **Little 5 Points Halloween Festival & Parade**: Features live music, an art and vintage market, ghost tours, a skate park, and food and drink vendors. [Learn more](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/)
3. **The Upside Down**: Event featuring celebrity panels, exclusive photo ops, and autograph sessions from celebrities from Stranger Things. [Learn more](https://www.epiccons.com/events/the-upside-down)
4. **Ellijay Apple Festival**: Continues through the weekend. [Learn more](https://www.atlantaonthecheap.com/fall-festivals-atlanta/)

Feel free to explore these events and make the most of your weekends in Atlanta!


In [6]:
using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.Formatting;

output.LastMessage.DisplayAs("text/markdown");


Here are some exciting events and things to do in Atlanta for each of the next four weekends:

**September 28-29, 2024:**
1. **Sweet Auburn Music Fest**: Located on Auburn Avenue, offering live music, international food court, national artists, celebrities, and vendors of all types. [Learn more](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-september-in-atlanta/)
2. **Riverfest**: An annual juried arts and crafts festival in Etowah River Park, Canton, with 150+ artists and artisans. [Learn more](https://gosouthatlanta.com/things-to-do/events/atlanta-in-september.html)
3. **East Atlanta Strut**: This iconic street festival celebrates East Atlanta’s culture, art, and quirkiness. [Learn more](https://www.atlantaonthecheap.com/fall-festivals-atlanta/)
4. **Concerts and Shows**: Various concerts and shows happening around Atlanta. [Learn more](https://atlanta-ga.events/september/)

**October 5-6, 2024:**
1. **Festival on Ponce**: Arts festival in Olmsted Linear Park with 125+ vendors, food, drink, and children’s activities. [Learn more](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html)
2. **Candler Park Fall Fest**: Includes an artist market, live acoustic music, rides, a tour of homes, food, and kids activities. [Learn more](https://www.atlantaonthecheap.com/fall-festivals-atlanta/)
3. **Atlanta Fair**: A traditional fair with cotton candy, a Ferris wheel, fishing for goldfish, and more. Located across from the Georgia State University Stadium. [Learn more](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/)
4. **Atlanta Horror Film Festival**: Held at Limelight Theater featuring the best indie horror films. [Learn more](https://www.experism.com/atlanta/october)

**October 12-13, 2024:**
1. **Ellijay Apple Festival**: Spend a day on the farm, pick your own apples, and visit over 100 vendor booths for arts, crafts, and apple-based food items. [Learn more](https://www.atlantaonthecheap.com/fall-festivals-atlanta/)
2. **Atlanta Pride Festival and Parade**: A vibrant event celebrating and supporting the LGBTQ+ community. [Learn more](https://unexpectedatlanta.com/atlanta-festival-calendar-2024/)
3. **Concerts and Shows**: Various concerts and shows happening around Atlanta. [Learn more](https://atlanta-ga.events/october/)
4. **Atlanta Fair**: Continues through October. [Learn more](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/)

**October 19-20, 2024:**
1. **PNC Atlanta 10 Miler & 5K**: Annual runs through Buckhead from Atlantic Station. [Learn more](https://gosouthatlanta.com/things-to-do/events/atlanta-in-october.html)
2. **Little 5 Points Halloween Festival & Parade**: Features live music, an art and vintage market, ghost tours, a skate park, and food and drink vendors. [Learn more](https://discoveratlanta.com/stories/things-to-do/cool-things-to-do-in-october-in-atlanta/)
3. **The Upside Down**: Event featuring celebrity panels, exclusive photo ops, and autograph sessions from celebrities from Stranger Things. [Learn more](https://www.epiccons.com/events/the-upside-down)
4. **Ellijay Apple Festival**: Continues through the weekend. [Learn more](https://www.atlantaonthecheap.com/fall-festivals-atlanta/)

Feel free to explore these events and make the most of your weekends in Atlanta!

## Clean Up

In [7]:
var assistant = (await client.AssistantList())?.Data?.FirstOrDefault(o => o.Name == "Web Search Test");
if(assistant != null) {
    await client.AssistantDelete(assistant.Id);
    Console.WriteLine("Deleted assistant");
}
else
{
    Console.WriteLine("Didn't find MsGraphUserProfile");
}

Deleted assistant
